In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat  = ChatOpenAI(model_name="gpt-4o-mini",
                temperature=0.1,
                streaming=True,
                callbacks=[StreamingStdOutCallbackHandler(),
                    ],
                )

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cusine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat


In [54]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a vegetarian chef specialized on making traditional recipes vegterian.
    You find alternative ingredients and explain their preparation.
    You don't radically modify the recipe.If there is no alternative for a food just say don't know how to replace it."""),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
    "cuisine" : "indian",
})


Great choice! Indian cuisine is rich in flavors and offers a variety of dishes. Here’s a simple recipe for **Chana Masala**, a popular vegetarian dish made with chickpeas. It’s flavorful, nutritious, and easy to prepare.

### Chana Masala Recipe

#### Ingredients:
- 2 cups canned chickpeas (or 1 cup dried chickpeas, soaked overnight and cooked)
- 2 tablespoons oil (vegetable or coconut)
- 1 large onion, finely chopped
- 2 tomatoes, chopped
- 2-3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 2 green chilies, slit (adjust to taste)
- 1 teaspoon cumin seeds
- 1 teaspoon coriander powder
- 1 teaspoon cumin powder
- 1 teaspoon garam masala
- ½ teaspoon turmeric powder
- Salt to taste
- Fresh cilantro, chopped (for garnish)
- Lemon wedges (for serving)

#### Instructions:

1. **Prepare the Chickpeas**: If using dried chickpeas, soak them overnight in water. Drain and rinse, then cook in boiling water until tender (about 1-1.5 hours). If using canned chickpeas, simply drain and rin

AIMessageChunk(content='### Vegetarian Chana Masala Recipe\n\nChana Masala is a delightful and nutritious dish that can easily be made vegetarian. Here’s how to prepare it with some alternative ingredients if needed.\n\n#### Ingredients:\n- **2 cups canned chickpeas** (or **1 cup dried chickpeas**, soaked overnight and cooked)\n- **2 tablespoons oil** (vegetable or coconut)\n- **1 large onion**, finely chopped\n- **2 tomatoes**, chopped\n- **2-3 cloves garlic**, minced\n- **1-inch piece of ginger**, grated\n- **2 green chilies**, slit (adjust to taste)\n- **1 teaspoon cumin seeds**\n- **1 teaspoon coriander powder**\n- **1 teaspoon cumin powder**\n- **1 teaspoon garam masala**\n- **½ teaspoon turmeric powder**\n- **Salt to taste**\n- **Fresh cilantro**, chopped (for garnish)\n- **Lemon wedges** (for serving)\n\n#### Instructions:\n\n1. **Prepare the Chickpeas**: \n   - If using dried chickpeas, soak them overnight in water. Drain and rinse, then cook in boiling water until tender (abou